In [23]:
import pandas as pd
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [24]:
gp_round = 1
dict_nb_laps = {1: 57, 2: 50, 3: 58, 4: 63, 5: 57, 6: 66, 7: 78, 8: 51, 9: 70, 10: 52, 11: 71, 12: 53, 13: 70, 14: 44, 15: 72, 16: 53, 17: 61, 18: 53, 19: 56, 20: 71, 21: 71, 22: 58}
dict_pit_stops_time = {1: 25, 2: 21, 3: 22, 4: 25, 5: 25, 6: 22, 7: 24, 8: 20, 9: 24, 10: 29, 11: 21, 12: 30, 13: 22, 14: 23, 15: 19, 16: 25, 17: 29, 18: 24, 19: 24, 20: 23, 21: 23, 22: 22}

In [25]:
# Course ??
tourTot = dict_nb_laps[gp_round]
perteArret = dict_pit_stops_time[gp_round]

In [26]:
df_combi = pd.DataFrame(columns=['Train1', 'Train2', 'Train3'])

for i in range(1,tourTot-1):
    T1 = i
    for j in range(1, tourTot-1):        
        if (i+j) > tourTot-1:
            continue
        T2 = j
        T3 = tourTot - j - i
        df_combi = df_combi.append({'Train1' : T1, 'Train2' : T2, 'Train3' : T3}, ignore_index = True)
        
df_combi

,Train1,Train2,Train3
0,1,1,55
1,1,2,54
2,1,3,53
3,1,4,52
4,1,5,51
...,...,...,...
1535,53,2,2
1536,53,3,1
1537,54,1,2
1538,54,2,1


In [27]:
df_combi2 = pd.DataFrame(columns=['Train1', 'Train2'])

for i in range(1,tourTot):
    T1 = i
    T2 = tourTot-i
    df_combi2 = df_combi2.append({'Train1' : T1, 'Train2' : T2}, ignore_index = True)
    
df_combi2.head()

,Train1,Train2
0,1,56
1,2,55
2,3,54
3,4,53
4,5,52


In [28]:
dffp_complet = pd.read_csv(f'https://f1-jedha-bucket.s3.eu-west-3.amazonaws.com/data/tyre_life_data_{gp_round}.csv', index_col=0)
dffp_complet.head()

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime,TimeStr
0,1,SOFT,97.308834,0.298077,0.067867,97.606911,97.539044,01:37.539
1,2,SOFT,97.308834,0.338318,0.135734,97.647152,97.511418,01:37.511
2,3,SOFT,97.308834,0.383990,0.203601,97.692824,97.489223,01:37.489
3,4,SOFT,97.308834,0.435829,0.271468,97.744663,97.473195,01:37.473
4,5,SOFT,97.308834,0.494666,0.339335,97.803500,97.464165,01:37.464


In [29]:
dffp_soft = dffp_complet[dffp_complet['Tyre']=='SOFT'].copy()
dffp_soft.drop(columns = ['Tyre', 'LapTimeSeconds', 'DeltaDeg', 'TimeStr', 'FinalLapTime'], inplace = True)
dffp_soft.rename(columns={'AdjustedTime': 'S', 'Lap': 'TyreLife'}, inplace = True)

dffp_medium = dffp_complet[dffp_complet['Tyre']=='MEDIUM'].copy()
dffp_medium.drop(columns = ['Tyre', 'LapTimeSeconds', 'DeltaDeg', 'TimeStr', 'FinalLapTime'], inplace = True)
dffp_medium.rename(columns={'AdjustedTime': 'M', 'Lap': 'TyreLife'}, inplace = True)

dffp_hard = dffp_complet[dffp_complet['Tyre']=='HARD'].copy()
dffp_hard.drop(columns = ['Tyre', 'LapTimeSeconds', 'DeltaDeg', 'TimeStr' ,'FinalLapTime'], inplace = True)
dffp_hard.rename(columns={'AdjustedTime': 'H', 'Lap': 'TyreLife'}, inplace = True)

In [30]:
dffp_final = dffp_soft.merge(dffp_medium, how = 'inner', left_on = 'TyreLife', right_on = 'TyreLife')
dffp_final = dffp_final.merge(dffp_hard, how = 'inner', left_on = 'TyreLife', right_on = 'TyreLife')
dffp_final.drop(columns = ['FuelDeg_y', 'FuelDeg_x'], inplace = True)
dffp_final = dffp_final[['TyreLife', 'S', 'M', 'H', 'FuelDeg']]

dffp_final

,TyreLife,S,M,H,FuelDeg
0,1,97.606911,98.707987,99.430675,0.067867
1,2,97.647152,98.732856,99.447225,0.135734
2,3,97.692824,98.760088,99.464850,0.203601
3,4,97.744663,98.789907,99.483621,0.271468
4,5,97.803500,98.822558,99.503612,0.339335
5,6,97.870280,98.858312,99.524902,0.407202
6,7,97.946075,98.897462,99.547577,0.475069
7,8,98.032103,98.940331,99.571725,0.542936
8,9,98.129744,98.987273,99.597443,0.610803
9,10,98.240567,99.038674,99.624832,0.678670


In [31]:
#Ouvre le csv avec les combinaisons pour 1 arrêts
tyreCombi2 = pd.read_csv('../data/TyreCombi2.csv', sep = ";")


#Ouvre le csv avec les combinaisons pour 2 arrêts
tyreCombi3 = pd.read_csv('../data/TyreCombi3.csv', sep = ";")

In [32]:
tyreCombi2

,P1,P2
0,S,M
1,S,H
2,M,S
3,M,H


In [33]:
tyreCombi3

,P1,P2,P3
0,S,S,M
1,S,M,M
2,M,M,S
3,M,S,S
4,S,M,S
5,S,H,S
6,S,S,H
7,S,H,H
8,S,M,H
9,S,H,M


In [34]:
# Boucle sur tous les types de pneu pour 2 arrêts afin de générer tous les temps totaux pour la course

time1 = 0
time2 = 0
time3 = 0
listeT = []
df_timerace3 = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeraceter = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timerace3 = df_timeraceter

In [35]:
#Boucle sur le df tyre_combi

for a in range(len(tyreCombi3)):
    lig = a
    df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])

    # Calcul du T1 en fonction du type de pneu 
    col = 0
    liste_combi = []
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
         
    #Boucle sur la longueur du df
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,0]
            
        #Addition des nbTours premières lignes
            
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time1 += dffp_final.iloc[i,colT]      
            
        df_timeracebis = df_timeracebis.append({'TimeT1' : time1}, ignore_index = True)
        df_timeracebis.iloc[l,1] = nbTour
        time1 = 0
        
        
# Calcul du T2 en fonction du type de pneu        
    col = 1
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
        
    #Boucle sur la longueur du df    
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,1]
        
        #Addition des nbTours premières lignes    
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time2 = time2 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,5] = time2
        df_timeracebis.iloc[l,2] = nbTour
        time2 = 0
        
        
# Calcul du T3 en fonction du type de pneu    
    
    col = 2
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
    
    #Boucle sur la longueur du df    
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,2]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time3 = time3 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,6]=time3
        df_timeracebis.iloc[l,3] = nbTour
        time3 = 0
        
        df_timeracebis.iloc[l,0] = "".join(liste_combi)

    df_timerace3 = pd.concat([df_timerace3,df_timeracebis], ignore_index = True)

In [36]:
#Calcul du temps total
df_timerace3['Total_time'] = df_timerace3['TimeT1'] + df_timerace3['TimeT2'] + df_timerace3['TimeT3'] + ( 2 * perteArret)
df_timerace3

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,SSM,1,1,55,97.606911,97.606911,5817.285651,6062.499474
1,SSM,1,2,54,97.606911,195.254063,5683.659089,6026.520063
2,SSM,1,3,53,97.606911,292.946887,5553.084703,5993.638501
3,SSM,1,4,52,97.606911,390.69155,5425.297693,5963.596155
4,SSM,1,5,51,97.606911,488.495051,5300.056233,5936.158195
...,...,...,...,...,...,...,...,...
23095,MHH,53,2,2,5553.084703,198.8779,198.8779,6000.840503
23096,MHH,53,3,1,5553.084703,298.34275,99.430675,6000.858128
23097,MHH,54,1,2,5683.659089,99.430675,198.8779,6031.967664
23098,MHH,54,2,1,5683.659089,198.8779,99.430675,6031.967664


In [37]:
time1 = 0
time2 = 0
time3 = 0
listeT = []
df_timerace2 = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracequa = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timerace2 = df_timeracequa

In [38]:
#Boucle sur le df tyre_combi

for a in range(0,len(tyreCombi2)):
    lig = a
    df_timeracebis = df_timeraceter

    # Calcul du T1 en fonction du type de pneu 
    col = 0
    liste_combi = []
    liste_combi.append(tyreCombi2.iloc[lig,col])
    
    if tyreCombi2.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi2.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
         
    #Boucle sur la longueur du df
    for l in range(len(df_combi2)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi2.iloc[l,0]
            
        #Addition des nbTours premières lignes
            
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time1 = time1 + dffp_final.iloc[i,colT]      
            
        df_timeracebis = df_timeracebis.append({'TimeT1' : time1}, ignore_index = True)
        df_timeracebis.iloc[l,1] = nbTour
        time1 = 0
        
        
# Calcul du T2 en fonction du type de pneu          
        
    col = 1
    liste_combi.append(tyreCombi2.iloc[lig,col])
    
    if tyreCombi2.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi2.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
        
    #Boucle sur la longueur du df    
    for l in range(len(df_combi2)):
        #nombre de tour égal à la valeur de la 2ème colonne
        nbTour = df_combi2.iloc[l,1]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time2 += dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,5] = time2
        df_timeracebis.iloc[l,2] = nbTour
        time2 = 0
        
        
# Génération du Train3 et Time3 à 0    
  
    
    #Boucle sur la longueur du df    
    for l in range(len(df_combi2)):        
            
        df_timeracebis.iloc[l,6]=0
        df_timeracebis.iloc[l,3] = 0
        
        
        df_timeracebis.iloc[l,0] = "".join(liste_combi)

    df_timerace2 = pd.concat([df_timerace2,df_timeracebis], ignore_index = True)

In [39]:
#Calcul du temsp total
df_timerace2['Total_time'] = df_timerace2['TimeT1'] + df_timerace2['TimeT2'] + df_timerace2['TimeT3']  + ( perteArret)
df_timerace2

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,SM,1,56,0,97.606911,5954.254348,0,6076.861259
1,SM,2,55,0,195.254063,5817.285651,0,6037.539714
2,SM,3,54,0,292.946887,5683.659089,0,6001.605976
3,SM,4,53,0,390.69155,5553.084703,0,5968.776253
4,SM,5,52,0,488.495051,5425.297693,0,5938.792744
...,...,...,...,...,...,...,...,...
219,MH,52,5,0,5425.297693,497.329983,0,5947.627676
220,MH,53,4,0,5553.084703,397.826371,0,5975.911074
221,MH,54,3,0,5683.659089,298.34275,0,6007.001838
222,MH,55,2,0,5817.285651,198.8779,0,6041.163551


In [40]:
#Concatène les 2 df
df_timerace2 = df_timerace2.sort_values(by='Total_time', ascending=True)

df_timerace3 = df_timerace3.sort_values(by='Total_time', ascending=True)


df_timerace = pd.concat([df_timerace2,df_timerace3], ignore_index = True)

# Affiche les 10 combinaisons les plus performantes
# df_timerace = df_timerace.sort_values('Total_time', ascending=True)

df_timerace

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,SH,21,36,0,2072.820903,3604.226036,0,5702.046939
1,SH,22,35,0,2174.388092,3502.74264,0,5702.130732
2,SH,20,37,0,1971.760215,3705.859408,0,5702.619623
3,SH,23,34,0,2276.530159,3401.400068,0,5702.930226
4,SH,19,38,0,1871.145782,3807.652505,0,5703.798287
...,...,...,...,...,...,...,...,...
23319,SMH,55,1,1,7687.362624,98.707987,99.430675,7935.501287
23320,SHM,55,1,1,7687.362624,99.430675,98.707987,7935.501287
23321,MSH,1,55,1,98.707987,7687.362624,99.430675,7935.501287
23322,MHS,1,1,55,98.707987,99.430675,7687.362624,7935.501287


In [41]:
total_fuel = dffp_final['FuelDeg'].sum()
df_timerace['Total_time'] = df_timerace['Total_time'] - total_fuel
df_timerace

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,SH,21,36,0,2072.820903,3604.226036,0,5589.862728
1,SH,22,35,0,2174.388092,3502.74264,0,5589.946522
2,SH,20,37,0,1971.760215,3705.859408,0,5590.435412
3,SH,23,34,0,2276.530159,3401.400068,0,5590.746016
4,SH,19,38,0,1871.145782,3807.652505,0,5591.614076
...,...,...,...,...,...,...,...,...
23319,SMH,55,1,1,7687.362624,98.707987,99.430675,7823.317076
23320,SHM,55,1,1,7687.362624,99.430675,98.707987,7823.317076
23321,MSH,1,55,1,98.707987,7687.362624,99.430675,7823.317076
23322,MHS,1,1,55,98.707987,99.430675,7687.362624,7823.317076


In [42]:
df_final = pd.concat([df_timerace[df_timerace['Train3'] == 0].iloc[:2], df_timerace[df_timerace['Train3'] != 0].iloc[:2]]).reset_index(drop=True).drop([1, 3], axis=0).reset_index(drop=True).sort_values(by='Total_time', ascending=True)
df_final

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
1,SMS,18,21,18,1770.924526,2083.146247,1770.924526,5562.811089
0,SH,21,36,0,2072.820903,3604.226036,0,5589.862728


In [43]:
df_final['TotalTimeStr'] = df_final['Total_time'].apply(lambda x: str(datetime.timedelta(seconds=x))[:-3])
df_final

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time,TotalTimeStr
1,SMS,18,21,18,1770.924526,2083.146247,1770.924526,5562.811089,1:32:42.811
0,SH,21,36,0,2072.820903,3604.226036,0,5589.862728,1:33:09.862


In [44]:
# df_final.to_csv(f'../data/predicted_strategy_round_{gp_round}.csv')